In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys

sys.path.append("../")

In [4]:
import fof.dataloader
import transformers as tr
from pathlib import Path

In [23]:
loader = fof.dataloader.ScicapDataModule(
  "First-Sentence",
  tokenizer=tr.AutoTokenizer.from_pretrained("distilbert-base-uncased"),
  root=Path("../scicap_data"),
  batch_size=1,
  num_workers=0)
train = loader.train_dataloader()

Initializing SCICAP training dataset
Initializing SCICAP testing dataset
Initializing SCICAP validation dataset
Time taken: 0.04568171501159668


In [31]:
batch = next(iter(train))
len(batch)

6

In [33]:
for key in ["references", "labels", "title", "id"]:
    print(key, batch[key])

references ['ons. All these codes fit in a broader class called alternant codes. See for\ninstance [82, Chap. 12, Fig. 12.1].\nIn some sense, algebraic geometry codes are natural generalisations of generalized\nReed–Solomo[SEP] q > 49, it is shown\nthat this TVZ bound beats the GV bound on a certain interval as illustrated in\nFigure 1\n– When q = p2m+1 (m > 1) is a non-prime odd power of a prime, Bassa, Beelen,\nGarcia, and Stichtenot']
labels ['Figure 1. Tsfasman–Vlăduţ–Zink bound for q = 64']
title ['Algebraic geometry codes and some applications']
id ['2009.01281v1-Figure1-1.png']
